Creates nice tables and stuff for the FITML'24 paper

In [ ]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [ ]:
import bokeh.layouts as bkl
import bokeh.plotting as bk
from bokeh.io import output_notebook

output_notebook()

# Load result files

In [ ]:
import turbo_broccoli as tb
import regex as re
from pathlib import Path

SWEEP_DIRS = [
    Path("out") / "sweep",
    Path("out") / "baselines",
]
SWEEP_RESULTS = []

for sweep_dir in SWEEP_DIRS:
    for p in sweep_dir.glob("**/results.*.json"):
        if not re.match(r"results\.\w+\.json", p.name):
            continue
        try:
            print("Loading", p)
            data = tb.load(p)
            SWEEP_RESULTS.append(data)
        except Exception as e:
            print("ERROR: Couldn't load", p, ":", type(e), str(e))

print()
print("Loaded", len(SWEEP_RESULTS), "results")

# Gather baselines

`baselines` is a dict

    dataset -> model_name -> ["acc1" or "acc5"] -> a float

In [ ]:
from collections import defaultdict
import numpy as np

BASELINE_OVERRIDES = {}

baselines = defaultdict(dict)
for r in SWEEP_RESULTS:
    if r["model"]["hparams"]["lcc_kwargs"] is not None:
        continue
    dataset, model = r["dataset"]["name"], r["model"]["name"]
    if model not in baselines[dataset]:
        baselines[dataset][model] = {"acc1": [], "acc5": []}
    baselines[dataset][model]["acc1"].append(r["training"]["test"][0]["test/acc"])
    baselines[dataset][model]["acc5"].append(r["training"]["test"][0]["test/acc5"])

for bls in baselines.values():
    for model, d in bls.items():
        bls[model]["acc1"] = np.max(bls[model]["acc1"])
        bls[model]["acc5"] = np.max(bls[model]["acc5"])

baselines

# Gather all results (incl. baselines) in a dataframe

In [ ]:
import pandas as pd
import numpy as np

from sweep import LCC_INTERVALS, LCC_WARMUPS, LCC_WEIGHTS, LCC_KS
from sweep import MODELS as _MODELS
from sweep import DATASETS as _DATASETS

MODELS = set(d["name"] for d in _MODELS)
DATASETS = set(d["name"] for d in _DATASETS)

GAIN_THRESHOLD = -10  # in percent

data = []

for ds, bsls in baselines.items():
    if ds not in DATASETS:
        continue
    for model, d in bsls.items():
        if model not in MODELS:
            continue
        data.append(
            {
                "model": model,
                "dataset": ds,
                "test_acc1": d["acc1"],
                "test_acc5": d["acc5"],
                # "gain1": 0,
                # "gain5": 0,
            }
        )

for r in SWEEP_RESULTS:
    if r["model"]["name"] not in MODELS or r["dataset"]["name"] not in DATASETS:
        # print(
        #     "Run not in model list or dataset list:",
        #     r["model"]["name"],
        #     r["dataset"]["name"],
        # )
        continue
    lcc_kwargs = r["model"]["hparams"]["lcc_kwargs"]
    lcc_submodules = r["model"]["hparams"].get("lcc_submodules")
    if lcc_submodules is None:  # Baseline run
        continue
    if not (
        lcc_kwargs is None
        or (
            lcc_kwargs["weight"] in LCC_WEIGHTS
            and lcc_kwargs["interval"] in LCC_INTERVALS
            and lcc_kwargs["warmup"] in LCC_WARMUPS
            and lcc_kwargs["k"] in LCC_KS
        )
    ):
        # print("Run LCC parameters now in sweep")
        continue
    if hasattr(lcc_submodules, "__len__") and len(lcc_submodules) > 1:
        # print("Run has more than one LCC submodule")
        continue

    row = {
        "model": r["model"]["name"],
        "dataset": r["dataset"]["name"],
        "test_acc1": r["training"]["test"][0]["test/acc"],
    }
    if (a5 := r["training"]["test"][0].get("test/acc5")) is not None:
        row["test_acc5"] = a5
    if lcc_kwargs is None:
        row["gain1"], row["gain5"] = 0, 0
    else:
        row["submodule"] = ",".join(lcc_submodules)
        row.update(lcc_kwargs)
        try:
            bacc1 = baselines[row["dataset"]][row["model"]]["acc1"]
            gain = row["test_acc1"] - bacc1
            row["gain1"] = gain
        except KeyError:
            row["gain1"] = 0
        try:
            bacc5 = baselines[row["dataset"]][row["model"]]["acc5"]
            gain = row["test_acc5"] - bacc5
            row["gain5"] = gain
        except KeyError:
            row["gain5"] = 0

    data.append(row)

df = pd.DataFrame(data)
# df = df.sort_values(columns, na_position="first")
# df

In [ ]:
columns = ["model", "dataset", "weight", "k", "interval", "warmup", "submodule"]
columns = [s for s in columns if s in df.columns]
df = df.groupby(columns, dropna=False).mean()
df = df.reset_index()
df = df.sort_values(columns, na_position="first")

interesting_columns = [
    c
    for c in df.columns
    if (df[c].isna().any() and len(df[c].unique()) > 2)
    or (not df[c].isna().any() and len(df[c].unique()) > 1)
]
if nic := [c for c in df.columns if c not in interesting_columns]:
    print("Dropping", nic)
    df = df.drop(nic, axis=1)

df

# Tex export

In [ ]:
N_DECIMALS = 2


def float_to_latex(x: float | str) -> str:
    if isinstance(x, str):
        return x
    if np.isnan(x):
        return ""
    float_str = f"{x:.2E}"
    if "E" in float_str:
        base, exponent = float_str.split("E")
        base, exponent = int(float(base)), int(exponent)
        if exponent < 0:
            if base == 1:
                return f"$10^{{{exponent}}}$"
            return f"${base} \\times 10^{{{exponent}}}$"
        return "$" + str(int(base * 10**exponent)) + "$"
    return "$" + str(round(x, N_DECIMALS)) + "$"


def int_to_latex(x: float | str) -> str:
    if isinstance(x, str):
        return x
    if np.isnan(x):
        return ""
    return "$" + str(int(x)) + "$"


def percent_to_latex(x: float | str, sign: bool = False) -> str:
    if isinstance(x, str):
        return x
    if np.isnan(x):
        return ""
    return (
        "$" + ("+" if sign and x > 0 else "") + str(round(x * 100, N_DECIMALS)) + "\\%$"
    )


data = []
for _, row in df.iterrows():

    row_fmt = {}

    if dataset := row.get("dataset"):
        if dataset == "cifar100":
            dataset = "CIFAR100"
        elif dataset == "timm/imagenet-1k-wds":
            dataset = "ImageNet"
        else:
            print("Unknown dataset:", dataset, "Skipping.")
            continue
        row_fmt["dataset"] = dataset

    if model := row.get("model"):
        model = row["model"]
        if model.startswith("google/mobilenet") or model.startswith("timm/mobilenet"):
            model = "MobileNet"
        elif model.startswith("microsoft/resnet-18") or model.startswith(
            "timm/resnet18"
        ):
            model = "ResNet18"
        elif model.startswith("google/vit-base-patch16-224"):
            model = "ViT"
        elif model.startswith("timm/tinynet"):
            model = "TinyNet"
        elif model.startswith("timm/vgg11"):
            model = "VGG11"
        elif model.startswith("timm/tf_efficientnet_l2"):
            model = "EfficientNet"
        elif model.startswith("timm/convnext_small"):
            model = "ConvNeXt"
        elif model.startswith("timm/inception_v3"):
            model = "InceptionV3"
        elif model == "alexnet":
            model = "AlexNet"
        else:
            print("Unknown model:", model, "Skipping.")
            continue
        row_fmt["Model"] = model

    layer = row.get("submodule")
    if layer is None or not isinstance(layer, str):
        layer = ""
    elif layer in ["classifier", "head", "fc"]:
        layer = "Head"
    else:
        layer = "$2^{\\text{nd}}$ to last"
        # layer = "$\\texttt{" + str(layer).replace("_", "\_") + "}$"
    row_fmt["Layer"] = layer

    if weight := row.get("weight"):
        row_fmt["$w$"] = float_to_latex(weight)

    if k := row.get("k"):
        row_fmt["$k$"] = int_to_latex(k)

    if (wmp := row.get("warmup")) is not None:
        row_fmt["Wmp."] = int_to_latex(row["warmup"])

    if interval := row.get("interval"):
        row_fmt["Int."] = int_to_latex(row["interval"])

    row_fmt.update(
        {
            "Acc. (top 1)": percent_to_latex(row["test_acc1"], sign=False),
            "Gain (top 1)": (
                percent_to_latex(row["gain1"], sign=True) if row["gain1"] else ""
            ),
            "Acc. (top 5)": percent_to_latex(row["test_acc5"], sign=False),
            "Gain (top 5)": (
                percent_to_latex(row["gain5"], sign=True) if row["gain5"] else ""
            ),
        }
    )
    data.append(row_fmt)

df_tex = pd.DataFrame(data)

In [ ]:
from IPython.display import display

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_tex)

In [ ]:
from pathlib import Path

OUTPUT_PATH = Path("out/papers/")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

In [ ]:
with (OUTPUT_PATH / "results-all.tex").open("w", encoding="utf-8") as fp:
    df_tex.to_latex(fp, index=False)

for model in df_tex["Model"].unique():
    with (OUTPUT_PATH / f"results-{model.lower()}.tex").open(
        "w", encoding="utf-8"
    ) as fp:
        a = df_tex[df_tex["Model"] == model]
        a.drop("Model", axis=1).to_latex(fp, index=False)

# Typst export

Reference: https://typst.app/docs/reference/model/table

In [ ]:
import re

header = [f"[*{c}*]" for c in data[0]]
n_columns = len(header)

typst = f"""
#figure(table(
  columns: ({', '.join(['auto'] * n_columns)}),
  inset: 15pt,
  align: horizon,
  table.header(
    {', '.join(header)}
  ),"""

for d in data:
    typst += "\n// ------------------------------"
    for k, v in d.items():
        if m := re.match(r"\$10\^{(.*)}\$", v):
            v = f"$10^({m.group(1)})$"
        elif m := re.match(r"\$(.*)\^{\\text{(.*)}}\$(.*)", v):
            a, b, c = m.groups()
            v = f'${a}^("{b}")${c}'
        v = v.replace("\\%", "%")
        typst += f"\n  [{v}], // {k}"

typst += "\n))"
print(typst)

In [ ]:
with (OUTPUT_PATH / "results-all.typ").open("w", encoding="utf-8") as fp:
    fp.write(typst)

# Converting result JSON schema

# Accuracy top 5